#**CHAPTER 07 앙상블 학습과 랜덤 포레스트**


##7.1 투표 기반 분류기



*   직접 투표 : 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측 (다수결 투표)

다수결 투표 분류기가 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다도 정확도가 높은 경우가 많다. 약한 학습기일지라도 충분하게 많고 다양하다면 앙상블은 높은 정확도를 내는 강한 학습기가 될 수 있다.


In [10]:
from sklearn.datasets import make_moons

X,y = make_moons(n_samples = 10000, noise = 0.4)

In [21]:
X.shape

(10000, 2)

In [24]:
X

array([[ 1.63441421, -0.70457056],
       [-0.08932533,  0.23436512],
       [ 1.91649693,  0.16352908],
       ...,
       [ 1.37509635, -0.60449646],
       [ 0.19503503,  0.39852302],
       [-0.45866247,  0.79522327]])

In [23]:
y.shape

(10000,)

In [11]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [25]:
X_train.shape

(8000, 2)

여러 분류기를 조합하여 사이킷런의 투표 기반 분류기(VotingClassifier)를 만들어 훈련시켜보자

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting = 'hard')
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [13]:
from sklearn.metrics import accuracy_score

for clf in (log_clf,rnd_clf,svm_clf, voting_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.827
RandomForestClassifier 0.8505
SVC 0.861
VotingClassifier 0.855


투표 기반 분류기가 다른 개별 분류기보다 성능이 조금 높다.

##7.2 배깅과 페이스팅





*   **배깅(bootstrap aggregating 의 줄임말)** : 훈련 세트에서 중복을 허용하여 샘플링하는 방식
*   **페이스팅(pasting)** : 중복을 허용하지 않고 샘플링하는 방식



수집함수는   
'분류' 일 때는 통계적 최빈값  
'회귀' 일 때는 평균 계산  

개별 예측기는 원본 훈련 세트로 훈련시킨 것보다 훨씬 크게 편향되어 있지만  
수집 함수를 통과하면 편향과 분산이 모두 감소한다. 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하지만 분산은 줄어든다.

###7.2.1 사이킷런의 배깅과 페이스팅

결정 트리 분류기 500개의 앙상블을 훈련시켜보자.  
각 분류기는 훈련 세트에서 중복을 허용하여 무작위로 선택된 100개의 샘플로 훈련된다.( 이는 배깅의 경우이고, 대신 페이스팅을 사용하려면 bootstrap = False로 지정하면 된다.) n_jogs 매개변수는 사이킷런이 훈련과 예측에 사용할 CPU 코어 수를 지정한다.



In [16]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    max_samples = 100, bootstrap = True, n_jobs = -1)

bag_clf.fit(X_train,y_train)
y_pred = bag_clf.predict(X_test)

###7.2.2 oob 평가

**oob(out-of-bag) 샘플** : 배깅 과정에서 선택되지 않은 훈련 샘플들  
예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않으므로 별도의 검증 세트를 사용하지 않고 oob 샘플을 사용해 평가할 수 있다.  
**앙상블의 평가는 각 에측기의 oob 평가를 평균하여 얻는다.**

In [17]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),n_estimators = 500,
    bootstrap = True, n_jobs= -1, oob_score = True)

bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

0.841875

oob 평가 결과를 보면 이 BaggingClassifier는 테스트 세트에서 약 84% 의 정확도를 얻을 것으로 보인다.

In [18]:
from sklearn.metrics import accuracy_score 
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.8445

테스트 세트에서 84% 의 정확도를 얻었다. 매우 비슷한 수치이다.



*   **oob_decision_function_** : oob 샘플에 대한 결정 함수의 값 확인  
각 훈련 샘플의 클래스 확률을 반환한다.(기반이 되는 예측기가 predict_proba() 메서드를 가지고 있기 때문에)



In [19]:
bag_clf.oob_decision_function_

array([[0.25988701, 0.74011299],
       [1.        , 0.        ],
       [1.        , 0.        ],
       ...,
       [0.66829268, 0.33170732],
       [0.        , 1.        ],
       [0.        , 1.        ]])

In [20]:
bag_clf.oob_decision_function_.shape

(8000, 2)

oob 평가는 첫 번째 훈련 샘플이 양성 클래스에 속할 확률을 74.01% 로 추정하고 있다.

##7.3 랜덤 패치와 랜덤 서브스페이스

BaggingClassifier는 특성 샘플링도 지원한다. 따라서 각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련된다.  

이 기법은 매우 고차원의 데이터셋을 다룰 때 유용하다.



*   **랜덤 패치 방식** : 훈련 특성과 샘플을 모두 샘플링하는 것
*   **랜덤 서브스페이스 방식** : 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것  

특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춘다.





##7.4 랜덤 포레스트

랜덤 포레스트는 일반적으로 배깅 방법(또는 페이스팅)을 적용한 결정 트리의 앙상블이다.

최대 16개의 리프 토드를 갖는 500개의 트리로 이뤄진 랜덤 포레스트 분류기를 여러 CPU 코어에서 훈련시켜보자

In [26]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators= 500, max_leaf_nodes = 16, n_jobs = -1)
rnd_clf.fit(X_train,y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [27]:
y_pred_rf

array([0, 0, 1, ..., 1, 1, 0])

랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 차즌 식으로 무작위성을 더 주입한다. 결국 트리를 더욱 다양하게 만들고 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만들어낸다.

BaggingClassifier를 사용해 앞의 RandomForestClassifier와 거의 유사하게 만들어 보자

In [31]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features ='auto', max_leaf_nodes = 16), n_estimators = 500, max_samples = 1.0, bootstrap =True, n_jobs=-1)

###7.4.2 특성 중요도

사이킷런은 어떤 특성을 사용한 노드가 (랜덤 포레스트에 있는 모든 트리에 걸쳐서) 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정한다.  
더 정확히 말하면 가중치 평균이며 각 노드의 가중치는 연관된 훈련 샘플 수와 같다.

사이킷런은 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결괏값을 정규화함.

In [33]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators =500,n_jobs = -1)
rnd_clf.fit(iris['data'],iris['target'])
for name,score in zip(iris['feature_names'],rnd_clf.feature_importances_):
    print(name,score)

sepal length (cm) 0.09876374559148975
sepal width (cm) 0.024927936342367733
petal length (cm) 0.4325520614507921
petal width (cm) 0.44375625661535056


가장 중요한 특성은 꽃잎의 길이(43%) 와 너비(43%) 이고 꽃받침의 길이와 너비는 비교적 덜 중요해보인다.

#7.5 부스팅

**부스팅** : 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법. 앞의 모델을 보완해가면서 일련의 예측기를 학습시킨다.

##7.5.1 에이다부스트

1. 알고리즘의 기반이 되는 첫번째 분류기를 훈련 세트에서 훈련시키고 예측을 만든다.  
2. 알고리즘이 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높인다.  
3. 두 번째 분류기는 업데이트된 가중치를 사용해 훈련 세트에서 훈련하고 다시 예측을 만든다.  
4. 다시 가중치를 업데이트한다.

200개의 아주 얕은 결정 트리를 기반으로 하는 에이다부스트 분류기를 훈련시켜보자. 여기서 사용하는 결정 트리는 max_depth = 1로 결정 노드 하나와 리프노드 두 개로 이루어진 트리이다.

In [34]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 1), n_estimators = 200,
    algorithm = 'SAMME.R',learning_rate= 0.5)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

###7.5.2 그레이디언트 부스팅

**그레이디언트 부스팅** : 이전 예측기가 만든 잔여 오차(residual error)에 새로운 예측기를 학습시킨다.

In [35]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

첫 번째 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor를 훈련시킨다.

In [37]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y)

DecisionTreeRegressor(max_depth=2)

두 번째 예측기가 만든 잔여 오차에 세 번째 회귀 모델을 훈련시킨다.

In [38]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

이제 세 개의 트리를 포함하는 앙상블 모델이 생겼다. 새로운 샘플에 대한 예측을 만들려면 모든 트리의 예측을 더하면 된다.

In [40]:
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1,tree_reg2,tree_reg3))

NameError: ignored

In [41]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators =3, learning_rate =1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)



*   learning_rate : 각 트리의 기여 정도<br>  
축소 기법: 0.1처럼 낮게 설정하면 앙상블을 훈련 세트에 학습시키기 위해 많은 트리가 필요하지만 일반적으로 예측의 성능은 좋아진다.



최적의 트리 수를 찾는 방법 -> **조기 종료 기법**

120개의 트리로 GBRT 앙상블을 훈련시키고 최적의 트리 수를 찾기 위해 각 훈련 단계에서 검증 오차를 측정한다. 마지막에 최적의 트리 수를 사용해 새로운 GBRT 앙상블을 훈련시킨다.

In [42]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train,X_val,y_train,y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2,n_estimators=120)
gbrt.fit(X_train,y_train)

errors = [mean_squared_error(y_val,y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators = bst_n_estimators)
gbrt_best.fit(X_train,y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=120)

실제로 훈련을 중지하는 방법으로 조기 종료를 구현할 수도 있다.  

warm_start = True 로 설정하면 사이킷런이 fit() 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 해준다. 다음 코드는 연속해서 다섯 번의 반복동안 검증 오차가 향상되지 않으면 훈련을 멈춘다.

In [45]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start = True)

min_val_error = float('inf')
error_going_up = 0
for n_estimators in range(1,120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train,y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val,y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break   #조기 종료

<확률적 그레이디언트 부스팅>

*   **subsample** : 각 트리가 훈련할 때 사용할 훈련 샘플의 비율 지정  
예를 들어 subsmaple = 0.25 라고 하면 각 트리는 무작위로 선택된 25%의 훈련 샘플로 학습된다. 편향이 높아지는 대신 분산이 낮아지게 된다. 또한 훈련 속도도 상당히 높아진다.



<XGBoost> : 쵲거화된 그레이디언트 부스팅 구현

In [46]:
import xgboost 
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
y_pred = xgb_reg.predict(X_val)

[05:33:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [48]:
xgb_reg.fit(X_train,y_train,eval_set=[(X_val,y_val)], early_stopping_rounds =2)
y_pred = xgb_reg.predict(X_val)

[05:48:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.47354
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.450841
[2]	validation_0-rmse:0.43156
[3]	validation_0-rmse:0.415299
[4]	validation_0-rmse:0.401226
[5]	validation_0-rmse:0.389469
[6]	validation_0-rmse:0.379336
[7]	validation_0-rmse:0.370766
[8]	validation_0-rmse:0.363586
[9]	validation_0-rmse:0.35768
[10]	validation_0-rmse:0.352678
[11]	validation_0-rmse:0.34838
[12]	validation_0-rmse:0.344557
[13]	validation_0-rmse:0.341534
[14]	validation_0-rmse:0.33889
[15]	validation_0-rmse:0.336888
[16]	validation_0-rmse:0.334906
[17]	validation_0-rmse:0.333266
[18]	validation_0-rmse:0.331862
[19]	validation_0-rmse:0.330816
[20]	validation_0-rmse:0.329794
[21]	validation_0-rmse:0.328656
[22]	validation_0-rmse:0.327826
[23]	validation_0-rmse:0.326933
[24]	validation_0-rmse:0.326097
[25]	validation_0-rmse:0